# Image Processing

The use of image processing to preprocess the image and convert it into a form suitable for further analysis. Examples of such operations include exposure correction and color balancing, reducing image noise, increasing sharpness, or straightening the image by rotating it.

This note contains following chapters:

    1. Point operators

    2. Linear filtering
    
    3. Neighborhood operators

    4. Fourier Transform

    5. Pyramids and Wavelets

    6. Geometric transformations

## Point operators

The simplest kinds of image processing transforms are point operators, where **each output pixel’s value depends on only the corresponding input pixel value**. Such operators include **brightness** and **contrast adjustments** as well as **color correction** and **transformations**. Such operations are also known as *point processes*.

This chapter contains following concepts:

    1.1 Pixel transforms

    1.2 Color transforms

    1.3 Compositing and Matting

    1.4 Histogram equalization

### Pixel transforms

A general image processing *operator* is a **function** that takes one or more input images and produces an output image:

$$
g(x) = h(f(x)) \text{ or } g(x) = h(f0(x), . . . , fn(x))
$$

where x is in the D-dimensional (usually D = 2 for images) domain of the input and output functions f and g, which operate over some range, which can either be scalar or vector- valued, e.g., for color images or 2D motion.

For discrete images, the domain consists of a finite number of pixel locations, x = (i, j), and we can write:

$$
g(i,j) = h(f(i,j))
$$

Two commonly used point processes are **multiplication** and **addition** with a **constant**:

$$
g(x) = af (x) + b
$$

The parameters a > 0 and b are often called the **gain** and **bias** parameters, sometimes these parameters are said to control **contrast** and **brightness**. The bias and gain parameters can also be **spatially varying**:

$$
g(x) = a(x)f (x) + b(x)
$$

Eg, when simulating the graded density filter used by photographers to *selectively darken the sky* or when *modeling vignetting in an optical system*.

Multiplicative gain (**both global and spatially varying**) is a *linear operation*, as it obeys the superposition principle:

$$
h(f_0 + f_1) = h(f_0) + h(f_1)
$$

Eg, Operators such as *image squaring* (which is often used *to get a local estimate of the energy in a bandpass filtered signal*) are not linear.

Another commonly used dyadic (two-input) operator is the **linear blend operator**:

$$
g(x) = (1 − α)f_0(x) + αf_1(x)
$$

By varying $α$ from 0 → 1, this operator can be used to perform a *temporal cross-dissolve* between two images or videos, as seen in slide shows and film production, or as a component of image morphing algorithms.

One highly used non-linear transform that is often applied to images before further processing is **gamma correction**, which is used to *remove the non-linear mapping between input radiance and quantized pixel values*. To invert the gamma mapping applied by the sensor, we can use:

$$
g(x) = [f(x)]^{1/γ}
$$

where a gamma value of $γ ≈ 2.2$ is a reasonable fit for most digital cameras.

### Color transforms

While color images can be treated as arbitrary vector-valued functions or collections of independent bands, it usually makes sense to think about them as highly correlated signals with strong connections to the *image formation process*, *sensor design*, and *human perception*. Consider, for example, brightening a picture by adding a constant value to all three channels, as shown in Figure 3.2b. Can you tell if this achieves the desired effect of making the image look brighter? Can you see any undesirable side-effects or artifacts?

![](image/cv_local_process.png)

In fact, adding the same value to each color channel not only **increases the apparent intensity of each pixel**, it can also affect the **pixel’s hue and saturation**. 

How can we define and manipulate such quantities in order to achieve the desired perceptual effects? **Chromaticity coordinates** or even simpler **color ratios** can first be computed and then used after manipulating (e.g., brightening) the luminance Y to re-compute a valid RGB image with the same hue and saturation. Figures 2.33 f - h show some color ratio images multiplied by the middle gray value for better visualization.

![](image/cv_color_sapce_transform.png)

Similarly, **color balancing** (e.g., to *compensate for incandescent lighting*) can be performed either by multiplying each channel with a different scale factor or by the more complex process of mapping to XYZ color space, changing the nominal white point, and mapping back to RGB, which can be written down using a linear 3 × 3 color twist transform matrix.

### Compositing and Matting

In many photo editing and visual effects applications, it is often desirable to cut a *foreground* object out of one scene and put it on top of a different *background* (Figure 3.4). The process of **extracting the object from the original image** is often called **matting**, while the process of **inserting it into another image** (without visible artifacts) is called **compositing**.

![](image/cv_compositing_matting.png)

The intermediate representation used for the foreground object between these two stages is called an **alpha-matted color image** (Figure 3.4 b–c). In addition to the three color RGB channels, an alpha-matted image contains a *fourth alpha channel* $α$ (or A) that describes the relative amount of *opacity or fractional* coverage at each pixel (Figures 3.4c and 3.5b). The opacity is the opposite of the transparency. Pixels within the object are fully opaque (α = 1), while pixels fully outside the object are *transparent (α = 0)*. Pixels on the boundary of the object vary smoothly between these two extremes, which hides the perceptual visible *jaggies* that occur if only binary opacities are used.

To composite a new (or foreground) image on top of an old (background) image, the **over operator** is used:

$$
C =(1−α)B+αF
$$

This *operator attenuates the influence of the background image* B by a factor $(1 − α)$ and then adds in the color (and opacity) values corresponding to the foreground layer F , as shown in Figure 3.5.

![](image/cv_over_opreator.png)

In many situations, it is convenient to represent the foreground colors in pre-multiplied form, i.e., to store (and manipulate) the $αF$ values directly. The pre-multiplied RGBA representation is preferred for several reasons, including the ability to blur or resample (e.g., rotate) alpha-matted images without any additional complications (just treating each RGBA band independently). However, when matting using local color consistency, the pure un-multiplied foreground colors F are used, since these remain constant (or vary slowly) in the vicinity of the object edge.

When light reflects off clean transparent glass, the light passing through the glass and the light reflecting off the glass are simply added together (Figure 3.6). This model is useful in the *analysis of transparent motion*, which occurs when such scenes are observed from a moving camera.

![](image/cv_light_reflect.png)

The actual process of matting, i.e., *recovering the foreground, background*, and *alpha matte values from one or more images*, has a rich history. It have a nice survey of traditional blue-screen matting techniques, that review difference matting. Since then, there has been a lot of activity in computational photography relating to natural image matting, which attempts to extract the mattes from a single natural image (Figure 3.4 a) or from extended video sequences.

### Histogram equalization

While the brightness and gain controls described can **improve the appearance of an image**, how can we automatically **determine their best values**? 

- One approach might be to **look at the darkest and brightest pixel values in an image and map them to pure black and pure white**. 

- Another approach might be to **find the average value in the image, push it towards middle gray, and expand the range so that it more closely fills the displayable values**.

How can we visualize the set of lightness values in an image to test some of these heuristics? The answer is to **plot the histogram of the individual color channels and luminance values**, as shown in Figure 3.7b. From this distribution, we can compute relevant statistics such as the *minimum*, *maximum*, and *average intensity* values. Notice that the image in Figure 3.7a has both an excess of dark values and light values, but that the mid-range values are largely under-populated. Would it not be better if we could simultaneously brighten some dark values and darken some light values, while still using the full extent of the available dynamic range? Can you think of a mapping that might do this?

![](image/cv_histogram_equalization.png)

One popular answer to this question is to perform **histogram equalization**, i.e., to find an intensity mapping function $f(I)$ such that the resulting *histogram is flat*. The trick to finding such a mapping is the **same** one that people use to generate random samples from a **probability density function (pdf)**, which is to first compute the *cumulative distribution function (cdf)* shown in Figure 3.7c.

Think of the original histogram h(I) as the distribution of grades in a class after some
exam. How can we map a particular grade to its corresponding percentile, so that students at the 75% percentile range scored better than 3/4 of their classmates? The answer is to integrate the distribution h(I) to obtain the cumulative distribution c(I),

$$
c(I) = \frac{1}{N} \sum^{I}_{i=0} h(i) = c(I − 1) + \frac{1}{N} h(I)
$$

where N is the number of pixels in the image or students in the class. For any given grade or intensity, we can look up its corresponding percentile c(I) and determine the final value that the pixel should take. When working with eight-bit pixel values, the I and c axes are rescaled from [0, 255].

Figure 3.7e shows the result of applying f(I) = c(I) to the original image. As we can see, the resulting *histogram is flat*; so is the resulting image (it is “flat” in the sense of a lack of contrast and being muddy looking). One way to compensate for this is to only partially compensate for the histogram unevenness, e.g., by using a mapping function $f(I) = αc(I) + (1 − α)I$, which is a linear blend between the cumulative distribution function (cdf) and the identity transform (a straight line). As you can see in Figure 3.7f, the resulting image maintains more of its original grayscale distribution while having a more appealing balance.

Another **potential problem** with histogram equalization (or, in general, image brightening) is that **noise in dark regions can be amplified and become more visible**. There are some possible ways to mitigate this, as well as **alternative techniques** to maintain contrast and “punch” in the original images.


#### Locally adaptive histogram equalization

While global histogram equalization can be useful, for some images it might be preferable to **apply different kinds of equalization in different regions**. Consider for example the image in Figure 3.8a, which has a wide range of luminance values. Instead of computing a single curve, what if we were to subdivide the image into M × M pixel blocks and perform separate histogram equalization in each sub-block? As you can see in Figure 3.8b, the resulting image exhibits a lot of blocking artifacts, i.e., intensity discontinuities at block boundaries.

![](image/cv_locally_adaptive_histogram_equalization.png)

- One way to eliminate blocking artifacts is to use a **moving window**, i.e., to recompute the histogram for every M × M block centered at each pixel. **This process can be quite slow** (M2 operations per pixel), although with clever programming only the histogram entries corresponding to the pixels entering and leaving the block (in a raster scan across the image) need to be updated (M operations per pixel).

- A more efficient approach is to compute **non-overlapped block-based equalization functions** as before, but to then smoothly interpolate the transfer functions as we move between blocks. 

This technique is known as **adaptive histogram equalization (AHE)** and its contrast- limited(gain-limited)version is known as CLAHE. The weighting function for a given pixel (i, j) can be computed as a function of its horizontal and vertical position (s, t) within a block, as shown in Figure 3.9a.

![](image/cv_adaptive_histogram_equalization.png)

To blend the four lookup functions $f_{00}, . . . , f_{11}$, a **bilinear blending function**:

$$
fs,t(I) = (1 − s)(1 − t)f_{00}(I) + s(1 − t)f_{10}(I) + (1 − s)tf_{01}(I) + st f_{11}(I)
$$

can be used. Note that instead of blending the four lookup tables for each output pixel (which would be quite slow), we can instead blend the results of mapping a given pixel through the four neighboring lookups.

A variant on this algorithm is to place the lookup tables at the corners of each M × M block (see Figure 3.9b and Exercise 3.8). In addition to blending four lookups to compute the final value, we can also distribute each input pixel into four adjacent lookup tables during the histogram accumulation phase (notice that the gray arrows in Figure 3.9b point both ways), i.e.,

$$
h_{k,l}(I(i, j)) += w(i, j, k, l)
$$

where w(i, j, k, l) is the bilinear weighting function between pixel (i, j) and lookup table (k, l). This is an example of soft histogramming, which is used in a variety of other applications, including the construction of SIFT feature descriptors and vocabulary trees.

## Linear filtering

Locally adaptive histogram equalization is an example of a **neighborhood operator** or **local operator**, which uses a collection of pixel values in the vicinity of a given pixel to determine its final output value (Figure 3.10).

![](image/cv_neighbourhood_filter_conv.png)

In addition to performing local tone adjustment, neighborhood operators can be used to filter images to add soft blur, sharpen details, accentuate edges, or remove noise (Figure 3.11b–d). In this section, we look at linear filtering operators, which involve fixed weighted combinations of pixels in small neighborhoods.

![](image/cv_nerighbour_operator.png)

The most widely used type of neighborhood operator is a linear filter, where an output pixel’s value is a weighted sum of pixel values within a small neighborhood N (Figure 3.10):

$$
g(i, j) = \sum_{k,l} f(i + k, j + l)h(k, l)
$$

The entries in the weight kernel or mask h(k, l) are often called the **filter coefficients**. The above correlation operator can be more compactly notated as:

$$
g = f ⊗ h
$$

A more common variant of the formula is:

$$
g(i, j) = \sum_{k,l} f(i − k, j − l)h(k, l) = \sum_{k,l} f(k, l)h(i − k, j − l)
$$

where the sign of the offsets in f has been reversed, This is called the **convolution operator**:

$$
g = f ∗ h
$$

and h is then called the **impulse response function**. The reason for this name is that the kernel function, h, convolved with an impulse signal, $δ(i, j)$ (an image that is 0 everywhere except at the origin) reproduces itself, $h ∗ δ = h$, whereas correlation produces the reflected signal.

In fact the variant formula g(i,j) can be interpreted as the superposition (addition) of shifted impulse response functions h(i − k, j − l) multiplied by the input pixel values f (k, l). Convolution has additional nice properties, e.g., it is both commutative and associative. As well, the Fourier transform of two convolved images is the product of their individual Fourier transforms.

Both correlation and convolution are linear shift-invariant (LSI) operators, which obey both the superposition principle:

$$
h◦(f0 +f1)=h◦f0 +h◦f1
$$

and the shift invariance principle:

$$
g(i,j)=f(i+k,j+l) ⇔ (h◦g)(i,j)=(h◦f)(i+k,j+l)
$$

which means that shifting a signal commutes with applying the operator ($◦$ stands for the LSI operator). Another way to think of shift invariance is that the operator “behaves the same everywhere”.

Occasionally, a shift-variant version of correlation or convolution may be used, Eg,

$$
g(i, j) = \sum_{k,l} f(i − k, j − l)h(k, l; i, j)
$$

where h(k, l; i, j) is the convolution kernel at pixel (i, j). For example, such a spatially varying kernel can be used to model blur in an image due to variable depth-dependent defocus.

Correlation and convolution can both be written as a matrix-vector multiplication, if we first convert the two-dimensional images f(i,j) and g(i,j) into raster-ordered vectors f and g:

$$
g = Hf
$$

where the (sparse) H matrix contains the convolution kernels. Figure 3.12 shows how a one-dimensional convolution can be represented in matrix-vector form.

![](image/cv_one_dimensional_convolution.png)

### Padding (border effects)

## Neighborhood operator

## Fourier transforms

## Pyramids and wavelets

## Geometric transforms